In [1]:
import os, math, random, argparse, ast, copy
from collections import Counter, defaultdict
from itertools import chain, combinations
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


### Local Imports

In [2]:
from utils.Spambase.split_data import split_data_equal
from utils.aggregate_functions import FederatedForest
from utils.DecisionTree import DecisionTree
from utils.nash1 import find_nash_equilibria_v2
from utils.evaluate_coalitions_new import evaluate_coalitions2

In [3]:
from utils.Nash_lottery import find_nash_equilibria_lottery

# HuGaDB Dataset 

In [5]:
import numpy as np

def payoff_10(mu: float, sigma: float) -> float:
    z = (
        1.9401
        + 0.0645 * mu
        + 0.0498 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_20(mu: float, sigma: float) -> float:
    z = (
        1.8267
        + 0.0649 * mu
        + 0.0419 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_30(mu: float, sigma: float) -> float:
    z = (
        1.6690
        + 0.0252 * mu
        + 0.0029 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_40(mu: float, sigma: float) -> float:
    z = (
        1.4980
        + 0.0333 * mu
        + 0.0124 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_50(mu: float, sigma: float) -> float:
    z = (
        1.3413
        + 0.0640 * mu
        - 0.0003 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_60(mu: float, sigma: float) -> float:
    z = (
        1.2171
        + 0.0983 * mu
        + 0.0198 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_70(mu: float, sigma: float) -> float:
    z = (
        1.0347
        + 0.1318 * mu
        + 0.0218 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_80(mu: float, sigma: float) -> float:
    z = (
        0.8202
        + 0.1994 * mu
        + 0.0243 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_90(mu: float, sigma: float) -> float:
    z = (
        0.6812
        + 0.2623 * mu
        - 0.0233 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_100(mu: float, sigma: float) -> float:
    z = (
        0.4823
        + 0.2904 * mu
        - 0.0194 * sigma
    )
    return 1 / (1 + np.exp(-z))

    z = (
        0.0000 * 1
        + 1.1246 * mu
        + 0.3975 * sigma
        - 0.9745 * mu**2
        + 0.3134 * mu * sigma
        - 0.6536 * sigma**2
        + 0.4823
    )
    return 1 / (1 + np.exp(-z))


In [4]:
def payoff_(mu: float, sigma: float) -> float:
    return         -0.0560 + 1.1664 * mu + 2.1711 * sigma

def payoff_100(mu: float, sigma: float) -> float:
    return         -1.4941+ 5.4708 * mu -0.7193 * sigma

In [5]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

In [6]:
# File patterns.
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata/train_{i:02d}.csv"
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata/test_{i:02d}.csv"

# Concatenate all training files.
df_train_global = pd.concat([
    pd.read_csv(train_files_pattern.format(i=i)) for i in range(1, 11)
]).dropna()

# Concatenate all testing files.
df_test_global = pd.concat([
    pd.read_csv(test_files_pattern.format(i=i)) for i in range(1, 11)
]).dropna()

# Split features and labels.
X_train_global = df_train_global.drop('act', axis=1)
y_train_global = df_train_global['act']

X_test_global = df_test_global.drop('act', axis=1)
y_test_global = df_test_global['act']

# Encode labels.
label_encoder = LabelEncoder()
y_train_global = label_encoder.fit_transform(y_train_global)
y_test_global = label_encoder.transform(y_test_global)

# Scale features.
scaler_global = StandardScaler()
X_train_global_scaled = scaler_global.fit_transform(X_train_global)
X_test_global_scaled  = scaler_global.transform(X_test_global)

In [9]:
# Parameters
eps               = 1e-8
n_clients_list    = [10,20,30,40,50,60,70,80,90,100]
n_trials          = 100
base_random_seed  = 42
max_depths        = [100]
approach          = 'fedfor'

# Saving directory
save_dir = "/Users/abbaszal/Documents/thimasfit11/hugadb_fedfor_all_test"
os.makedirs(save_dir, exist_ok=True)



train_csv_path = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata/new_runs/train.csv"
df_full_train = pd.read_csv(train_csv_path).dropna(subset=['act']).reset_index(drop=True)

for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")
          
    # Dynamically get the corresponding payoff function
    #payoff_func_name = f"payoff_{n_clients}"
    #if payoff_func_name not in globals():
        #raise ValueError(f"Missing payoff function for {n_clients} clients")
    #payoff_func = globals()[payoff_func_name]
 

    for max_depth in max_depths:
        print(f"\n max_depth = {max_depth}")

        # reset counters for this config
        counts_static = Counter()
        lottery_count = 0

        for trial in range(1, n_trials + 1):
            
            rand_component = random.randint(0, 500)
            trial_seed = base_random_seed + trial + int(1000 * max_depth) + 2 * rand_component



            X_test_full_scaled = X_test_global_scaled.copy()
            y_test_full        = y_test_global.copy()

            # Create a stratified subsample of the test set to speed up the runtime.
            subsample_size = 950  
            X_test_glob, _, y_test_glob, _ = train_test_split(
                X_test_full_scaled, y_test_full,
                train_size=subsample_size,
                random_state=trial_seed,
                stratify=y_test_full
            )



            df_trial, _ = train_test_split(
                df_full_train,
                train_size=10000,
                random_state=trial_seed,
                stratify=df_full_train['act']
            )
   
            df_remaining = df_trial.copy()


            client_partitions = []

            sample_size = 5000 // n_clients
            for client_idx in range(n_clients):
                if len(df_remaining) < sample_size:
                    raise ValueError("err")

                if len(df_remaining) == sample_size:
                    df_client = df_remaining.copy()
                    df_remaining = df_remaining.iloc[0:0] 

                else:
                    df_client, df_remaining = train_test_split(
                        df_remaining,
                        train_size=sample_size,
                        random_state=trial_seed,
                        stratify=df_remaining['act']
                    )
                df_client = df_client.reset_index(drop=True)
                client_partitions.append(df_client)

            # train each client
            client_models = []
            client_global_accuracies   = {}

            for client_idx, df_client in enumerate(client_partitions):

                X_client = df_client.drop(columns=['act'])
                y_client = df_client['act']

                client_scaler = StandardScaler()
                X_client_scaled = client_scaler.fit_transform(X_client)
                y_client_encoded = label_encoder.transform(y_client)
                # fit model
                model = DecisionTree(
                    max_depth=max_depth,
                    random_state=trial_seed
                )
                model.fit(X_client_scaled, y_client_encoded)
                client_models.append(model)

                # record its global-test accuracy
                y_pred_global = model.predict(X_test_glob)
                acc_global = accuracy_score(y_test_glob, y_pred_global)
                client_global_accuracies[client_idx] = acc_global

            # coalition evaluation
            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_global_accuracies,
                n_clients=n_clients,
                aggregator_func=FederatedForest,
                X_test=X_test_glob,
                y_test=y_test_glob,
                corrupt_client_indices=[],
                approach=approach
            )

            # static‐game Nash counts
            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_static[coalition] += 1


            # lottery‐game incentive check
            vals      = np.array(list(client_global_accuracies.values()))
            mu_full   = vals.mean()
            sig_full  = vals.std(ddof=1)
            payoff_f  = payoff_100(mu_full, sig_full)
            print(payoff_f)

            has_incentive = any(
                acc > payoff_f + eps
                for acc in client_global_accuracies.values()
            )
            if not has_incentive:
                lottery_count += 1


        static_count = sum(counts_static.values())

        counts_df = pd.DataFrame([{
            'n_clients':           n_clients,
            'max_iter':            max_depth,
            'Static_Occurrences':  static_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = (f"Nash_Counts_{approach}"
                 f"_nclients_{n_clients}"
                 f"_maxiter_{max_depth}.csv")
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)
        print(f"saved {fname}")



> n_clients = 10

 max_depth = 100
2.2908856839085243
2.251982725191321
2.4286322385256924
2.3522601419029328
2.4291737566630474
2.517832336050205
2.226604901667627
1.9458156148583599
2.2586087060363713
2.1077426902626097
2.27599922732894
2.415957657571037
2.303649485131151
2.4108092848057425
2.352311426393337
2.299408117789389
2.152744725591641
2.3148318688675773
2.2586087060363713
2.2372143049266366
2.100641543450848


KeyboardInterrupt: 

# Spambase Dataset

In [4]:
file_path = '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/spambase.data'  # Adjust the path as needed
df = pd.read_csv(file_path, header=None)

In [6]:
import numpy as np

def payoff_10(mu: float, sigma: float) -> float:
    z = (
        2.4879
        + 0.0813 * mu
        + 0.0032 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_20(mu: float, sigma: float) -> float:
    z = (
        2.4693
        + 0.0771 * mu
        + 0.0182 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_30(mu: float, sigma: float) -> float:
    z = (
        2.4729
        + 0.0798 * mu
        + 0.0237 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_40(mu: float, sigma: float) -> float:
    z = (
        2.4175
        + 0.0708 * mu
        + 0.0253 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_50(mu: float, sigma: float) -> float:
    z = (
        2.3852
        + 0.0739 * mu
        + 0.0278 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_60(mu: float, sigma: float) -> float:
    z = (
        2.3613
        + 0.0779 * mu
        + 0.0227 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_70(mu: float, sigma: float) -> float:
    z = (
        2.3083
        + 0.0750 * mu
        + 0.0293 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_80(mu: float, sigma: float) -> float:
    z = (
        2.2915
        + 0.0820 * mu
        + 0.0289 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_90(mu: float, sigma: float) -> float:
    z = (
        2.2969
        + 0.0813 * mu
        + 0.0241 * sigma
    )
    return 1 / (1 + np.exp(-z))

def payoff_100(mu: float, sigma: float) -> float:
    z = (
        2.2662
        + 0.0648 * mu
        + 0.0289 * sigma
    )
    return 1 / (1 + np.exp(-z))


In [ ]:
def payoff_10(mu: float, sigma: float) -> float:
    return 0.8734 * mu + 0.0597 * sigma + 0.1711

def payoff_20(mu: float, sigma: float) -> float:
    return 0.8152 * mu + 0.3268 * sigma + 0.2298

def payoff_30(mu: float, sigma: float) -> float:
    return 0.8610 * mu + 0.4550 * sigma + 0.1987

def payoff_40(mu: float, sigma: float) -> float:
    return 0.8319 * mu + 0.5583 * sigma + 0.2237

def payoff_50(mu: float, sigma: float) -> float:
    return 0.8872 * mu + 0.5625 * sigma + 0.1826

def payoff_60(mu: float, sigma: float) -> float:
    return 0.9809 * mu + 0.5364 * sigma + 0.1121

def payoff_70(mu: float, sigma: float) -> float:
    return 0.8383 * mu + 0.6407 * sigma + 0.2236

def payoff_80(mu: float, sigma: float) -> float:
    return 0.9885 * mu + 0.7051 * sigma + 0.1043

def payoff_90(mu: float, sigma: float) -> float:
    return 1.0581 * mu + 0.5148 * sigma + 0.0617

def payoff_100(mu: float, sigma: float) -> float:
    return 0.8793 * mu + 0.7293 * sigma + 0.1931

In [5]:
def payoff_(mu: float, sigma: float) -> float:
    return 0.5111 * mu + 0.8448 * sigma + 0.4752

def payoff_100(mu: float, sigma: float) -> float:
    return 0.8793 * mu + 0.7293 * sigma + 0.1931

In [6]:
def train_models_fedfor(partitions, X_test, y_test, max_depth):
    client_models = []
    client_global_accuracies = {}
    
    for i, (X_i, y_i) in enumerate(partitions):
        local_scaler = StandardScaler()
        model = DecisionTreeClassifier(max_depth=max_depth, random_state=np.random.randint(0, 100000))
        model.fit(local_scaler.fit_transform(X_i), y_i)
        client_models.append(model)
        y_pred = model.predict(X_test)
        client_global_accuracies[i] = np.mean(y_pred == y_test)
    return client_models, client_global_accuracies

In [ ]:
# Parameters
eps     = 1e-8
n_clients_list    = [10,20,30,40,50,60,70,80,90,100]
n_trials    = 100
base_seed   = 42
max_depths   = [ 100]
approach    = 'fedfor'


save_dir = (
"/Users/abbaszal/Documents/thimasfit11/spambase_fedfor_all_test"
)
os.makedirs(save_dir, exist_ok=True)

all_results = [] 


for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")
          
    # Dynamically get the corresponding payoff function
    #payoff_func_name = f"payoff_{n_clients}"
    #if payoff_func_name not in globals():
        #raise ValueError(f"Missing payoff function for {n_clients} clients")
    #payoff_func = globals()[payoff_func_name]
 

    for max_depth in max_depths:
        print(f"\n max_depth = {max_depth}")
        counts_static = Counter()
        lottery_count = 0

        for trial in range(1, n_trials+1):
            rc = random.randint(0, 500)
            trial_seed = base_seed + (trial-1) + 1000*max_depth + 2*rc
            random.seed(trial_seed)
            np.random.seed(trial_seed)

            X = df.iloc[:, :-1].to_numpy()
            y = df.iloc[:, -1].to_numpy()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=trial_seed)

            scaler = StandardScaler()

            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            partitions = split_data_equal(
                X_train, y_train,
                n_clients=n_clients,
                shuffle=True,
                random_seed=trial_seed
            )
            client_models, client_accs = train_models_fedfor(
                partitions=partitions,
                X_test=X_test,
                y_test=y_test,
                max_depth=max_depth
            )

            # coalition evaluation
            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_accs,
                n_clients=n_clients,
                aggregator_func=FederatedForest,
                X_test=X_test,
                y_test=y_test,
                corrupt_client_indices=[],
                approach=approach
            )

            # static‐game Nash counts
            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_static[coalition] += 1


            # lottery‐game incentive check
            vals = np.array(list(client_accs.values()))
            mu_full   = vals.mean()
            sig_full  = vals.std(ddof=1)
            payoff_f  = payoff_100(mu_full, sig_full)
            print(payoff_f)

            has_incentive = any(
                acc > payoff_f 
                for acc in client_accs.values()
            )
            if not has_incentive:
                lottery_count += 1

        # aggregate counts into single numbers
        static_count = sum(counts_static.values())


        counts_df = pd.DataFrame([{
            'n_clients':           n_clients,
            'max_iter':            max_depth,
            'Static_Occurrences':  static_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = (f"Nash_Counts_{approach}"
                    f"_nclients_{n_clients}"
                    f"_maxiter_{max_depth}.csv")
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)
        print(f" saved {fname}")


> n_clients = 10

 max_depth = 100
0.9623248542458753
0.9339673443440002
0.9606133138797905
0.9616829227900301
0.946406412674474
0.9588769041391265
0.9541529527694068
0.9700721699661525
0.9473552157597886
0.9697326643024966
0.9420102880598298
0.9613095404296204
0.972906004597526
0.9540580576711901
0.998563203521843
0.9383230190366829
0.9459549779503658
0.9761289613096853
0.9731028955339478
0.9617505525134504
0.9648612979137867
0.9614630735654064
0.9624598107768241
0.9587349749994265
0.9443663235989821
0.9911763494879897
0.9618233908346623
0.9811517739543376
0.9825582714262482
0.9822647357067131
0.9245776911181371
0.9854481306242051
0.9579896576605733
0.960150933283862
0.9701818063027956
0.9549278024888066
0.9643703659295759
0.9501340295190643
0.9565532737899101
0.9662585839887097
0.9745588967891907
0.9685980331573956
0.9674476510490635
0.9613095825750568
0.9565063433892487
0.9725063038532333
0.9747542964524607
0.94177938227831
0.9573196426584016
0.9669245781040985
0.9628375049005025
0